In [35]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
from PIL import Image
import os
from keras.datasets import cifar10

In [36]:
# initialize the data and labels
data = []
labels = []

In [37]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('/home/pratik/Desktop/images/')))
random.seed(42)
random.shuffle(imagePaths)

In [38]:
imagePaths[:5]

['/home/pratik/Desktop/images/baseball/images (3).jpeg',
 '/home/pratik/Desktop/images/baseball/images (1).png',
 '/home/pratik/Desktop/images/baseball/download (4).jpeg',
 '/home/pratik/Desktop/images/Football/images (6).jpeg',
 '/home/pratik/Desktop/images/Football/download.jpeg']

In [39]:
for imagePath in imagePaths:
	image = cv2.imread(imagePath)
    # flatten the image into 32x32x3=3072
	image = cv2.resize(image, (32, 32)).flatten()
	data.append(image) 
	data.append(image) #Append one more level
	# extract the class label from the image path and update the labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
	labels.append(label) #appended one more label

In [40]:
data[:1][0]

array([252, 123,  28, ...,  63,  36,  15], dtype=uint8)

In [41]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [42]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

In [43]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [44]:
# define the 3072-1024-256-3 architecture using Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
#model.add(Dense(64, activation="sigmoid")) #Hidden 1
model.add(Dense(128, activation="sigmoid")) #Hidden 2
model.add(Dense(3, activation="softmax")) #How many predictions do you want?

In [47]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.03
EPOCHS = 60
 
# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] training network...


In [48]:
# train the neural network
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=16)

Train on 75 samples, validate on 25 samples
Epoch 1/60
75/75 [==============================] - 2s 24ms/step - loss: 0.6992 - acc: 0.7733 - val_loss: 0.7831 - val_acc: 0.6800
Epoch 2/60
75/75 [==============================] - 0s 4ms/step - loss: 0.6729 - acc: 0.8400 - val_loss: 0.8338 - val_acc: 0.6400
Epoch 3/60
75/75 [==============================] - 0s 4ms/step - loss: 0.6844 - acc: 0.7733 - val_loss: 0.7862 - val_acc: 0.6400
Epoch 4/60
75/75 [==============================] - 0s 4ms/step - loss: 0.6905 - acc: 0.8000 - val_loss: 0.8564 - val_acc: 0.5600
Epoch 5/60
75/75 [==============================] - 0s 4ms/step - loss: 0.6438 - acc: 0.8267 - val_loss: 0.9043 - val_acc: 0.5200
Epoch 6/60
75/75 [==============================] - 0s 4ms/step - loss: 0.6611 - acc: 0.8133 - val_loss: 0.7537 - val_acc: 0.7200
Epoch 7/60
75/75 [==============================] - 0s 4ms/step - loss: 0.6230 - acc: 0.8267 - val_loss: 0.7574 - val_acc: 0.8000
Epoch 8/60
75/75 [===========================

In [49]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [50]:
class SymNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1
        
        # CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
         # (CONV => RELU) * 2 => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
        # first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.4))
        
		model.add(Dense(128))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.3))


		# softmax classifier
		model.add(Dense(5))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [51]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

symnet = SymNet()

In [52]:
# initialize the data and labels
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('/home/pratik/Desktop/images')))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
	# load the image, resize it to 64x64 pixels (the required input
	# spatial dimensions of SmallVGGNet), and store the image in the
	# data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (64,64))
	data.append(image)
	data.append(image)

	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
	labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [53]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [54]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# initialize our VGG-like Convolutional Neural Network
model = symnet.build(width=64, height=64, depth=3,
	classes=len(lb.classes_))

In [55]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.03
EPOCHS = 45
BS = 7
# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

[INFO] training network...


ValueError: Error when checking target: expected activation_18 to have shape (5,) but got array with shape (3,)

In [57]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('/home/pratik/Desktop/images/symnet_results.png')

# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save('symnet.model')
f = open('labels.bin', "wb")
f.write(pickle.dumps(lb))
f.close()


[INFO] evaluating network...
              precision    recall  f1-score   support

    Football       0.28      1.00      0.44         7
    baseball       0.00      0.00      0.00        12
cricket_ball       0.00      0.00      0.00         6

   micro avg       0.28      0.28      0.28        25
   macro avg       0.09      0.33      0.15        25
weighted avg       0.08      0.28      0.12        25



/home/pratik/anaconda3/envs/ann/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ValueError: x and y must have same first dimension, but have shapes (45,) and (60,)